In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pancancer-8-dataset/genes.csv
/kaggle/input/pancancer-8-dataset/8_cancers_merged.csv
/kaggle/input/pancancer-8-dataset/Preprocessed_8_cancer_genes.csv
/kaggle/input/autoencoder-features/top_0.25_percent_features_cv.csv
/kaggle/input/autoencoder-features/top_0.5_percent_features_cv.csv
/kaggle/input/autoencoder-features/top_1.0_percent_features_cv.csv


In [2]:
col_1 = list(pd.read_csv('/kaggle/input/autoencoder-features/top_0.25_percent_features_cv.csv').columns)
col_2 = list(pd.read_csv('/kaggle/input/autoencoder-features/top_0.5_percent_features_cv.csv').columns)
col_3 = list(pd.read_csv('/kaggle/input/autoencoder-features/top_1.0_percent_features_cv.csv').columns)

In [3]:
df = pd.read_csv('/kaggle/input/pancancer-8-dataset/Preprocessed_8_cancer_genes.csv')
df = df.iloc[:, 1:]
temp = df.iloc[:, :-1]
null_rows = temp[temp.isna().all(axis=1)].index
print(null_rows)
df.drop(index=[1226, 1741, 1917, 2069, 2494, 3084, 3637], inplace=True)
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]

Index([1226, 1741, 1917, 2069, 2494, 3084, 3637], dtype='int64')


In [4]:
X_actual = X

In [5]:
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import csv

# Encode the target variable (if it's categorical)
y = pd.factorize(Y)[0]
i = 1

for col in [col_1, col_2, col_3]:
        
    X = X_actual[col]
    
    # Split data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Genetic Algorithm setup with two objectives: accuracy and feature count
    creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))  # Maximize accuracy, minimize feature count
    creator.create("Individual", list, fitness=creator.FitnessMulti)
    
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", np.random.randint, 0, 2)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(X.columns))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # Fitness function using Random Forest
    def evaluate_rf(individual, X_train, y_train):
        # Select the features that are '1' in the individual
        selected_features = [index for index, value in enumerate(individual) if value == 1]
        
        if len(selected_features) == 0:
            return -np.inf,  # No features selected, so return a very low fitness score
        
        X_train_selected = X_train.iloc[:, selected_features]
        
        # Random Forest classifier with cross-validation to evaluate model performance
        rf = RandomForestClassifier(random_state=42, n_jobs=-1)
        scores = cross_val_score(rf, X_train_selected, y_train, cv=2, scoring='accuracy')
    
        accuracy = np.mean(scores) 
        
        # Return the accuracy and the feature count
        return accuracy, len(selected_features)
    
    # GA Operators
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selNSGA2)  # Pareto dominance selection
    toolbox.register("evaluate", evaluate_rf, X_train=X_train, y_train=y_train)

    # Determine population size based on feature count
    popsize = 50
    
    # Algorithm: Genetic Algorithm for Feature Selection
    population = toolbox.population(n=popsize)
    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=20, verbose=True)
    
    # Get the Pareto front (best solutions in the population)
    pareto_front = tools.sortNondominated(population, len(population), first_front_only=True)[0]
    
    # Display the Pareto front
    print("Pareto Front:")
    for ind in pareto_front:
        print(f"Accuracy: {ind.fitness.values[0]}, Feature Count: {ind.fitness.values[1]}")
        selected_features = [index for index, value in enumerate(ind) if value == 1]
        #print("Selected features:", X.columns[selected_features])
    
    # Choose the best individual based on your trade-off between accuracy and feature count
    best_individual = pareto_front[0]  # Choose the one that gives the best balance between accuracy and complexity
    selected_features = [index for index, value in enumerate(best_individual) if value == 1]
    print("feature set: {}".format(i))
    selected_gene_names = X.columns[selected_features].tolist()
    print("Selected gene names:", selected_gene_names)

    # Define the filename
    file_name = f"NSGA2_{len(X.columns)}_compression_{i}.csv"
    
    # Save the selected gene names to a CSV file
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Gene Name"])  # Header row
        for gene_name in selected_gene_names:
            writer.writerow([gene_name])
    
    print(f"Selected gene names saved to {file_name}")


    i += 1

gen	nevals
0  	50    
1  	32    
2  	43    
3  	41    
4  	31    
5  	37    
6  	30    
7  	38    
8  	41    
9  	32    
10 	35    
11 	34    
12 	42    
13 	40    
14 	40    
15 	37    
16 	46    
17 	45    
18 	42    
19 	40    
20 	37    
Pareto Front:
Accuracy: 0.936950783849603, Feature Count: 35.0
Accuracy: 0.927201002626811, Feature Count: 32.0
Accuracy: 0.9342085535022538, Feature Count: 34.0
Accuracy: 0.9387781951678257, Feature Count: 41.0
Accuracy: 0.9028350854066138, Feature Count: 30.0
feature set: 1
Selected gene names: ['AL627309.5', 'MTATP8P1', 'WASH7P', 'MTATP6P1', 'AL732372.3', 'MIR6859-2', 'AL669831.6', 'OR4F29', 'LINC02593', 'AL645608.2', 'AL627309.2', 'AL669831.3', 'MTCO3P12', 'ISG15', 'CICP7', 'MTND2P28', 'AL645608.6', 'AL669831.1', 'OR4F5', 'U6', 'WBP1LP6', 'AGRN', 'AL645608.1', 'CICP27', 'FAM41C', 'KLHL17', 'WASH9P', 'RNU6-1100P', 'SAMD11', 'AL390719.1', 'LINC01409', 'AL669831.4', 'AL627309.3', 'AL627309.4', 'AC114498.2']
Selected gene names saved to NSGA2_77_co

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	50    
1  	36    
2  	39    
3  	38    
4  	37    
5  	43    
6  	38    
7  	37    
8  	40    
9  	39    
10 	41    
11 	46    
12 	32    
13 	39    
14 	41    
15 	40    
16 	39    
17 	32    
18 	45    
19 	40    
20 	40    
Pareto Front:
Accuracy: 0.9543110058110492, Feature Count: 71.0
Accuracy: 0.9421272121734393, Feature Count: 56.0
Accuracy: 0.9533980423986147, Feature Count: 66.0
Accuracy: 0.9470021027848353, Feature Count: 60.0
feature set: 2
Selected gene names: ['SAMD11', 'SSU72', 'CDK11B', 'MRPL20-DT', 'AL627309.3', 'AL645608.7', 'RN7SL657P', 'KLHL17', 'RNF223', 'AL669831.1', 'CICP7', 'AL645608.6', 'TNFRSF4', 'CFAP74', 'FO538757.1', 'CICP3', 'LINC01342', 'MIR6727', 'MIR1302-2', 'AL627309.2', 'MXRA8', 'OR4F29', 'OR4F5', 'VWA1', 'MTND1P23', 'AL645608.3', 'FAAP20', 'NADK', 'NOC2L', 'TTLL10-AS1', 'MMP23A', 'MIR6726', 'DDX11L1', 'LINC01770', 'DVL1', 'CDK11A', 'AL513477.1', 'AL669831.5', 'WBP1LP6', 'AL669831.3', 'FO704657.1', 'WASH7P', 'UBE2J2', 'MIR6808', 'HES4', 

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	50    
1  	47    
2  	38    
3  	38    
4  	35    
5  	31    
6  	39    
7  	42    
8  	41    
9  	37    
10 	38    
11 	30    
12 	38    
13 	41    
14 	32    
15 	41    
16 	43    
17 	37    
18 	38    
19 	41    
20 	37    
Pareto Front:
Accuracy: 0.9600988598348799, Feature Count: 140.0
Accuracy: 0.9600979320265338, Feature Count: 132.0
Accuracy: 0.9530933501377981, Feature Count: 129.0
Accuracy: 0.9610120088089835, Feature Count: 163.0
Accuracy: 0.9613178144398153, Feature Count: 176.0
feature set: 3
Selected gene names: ['AL590822.1', 'MIR4251', 'ZBTB48', 'AL645728.2', 'AJAP1', 'AL031848.2', 'TAS1R1', 'AL627309.3', 'AL034417.4', 'AL589702.1', 'AL139415.1', 'DFFB', 'PRDM16-DT', 'AL512330.1', 'TTC34', 'SLC35E2B', 'CAMTA1-AS2', 'AC114498.1', 'SDF4', 'CA6', 'PLCH2', 'Z98747.1', 'Z98259.2', 'AL359881.3', 'CDK11A', 'OR4F5', 'AURKAIP1', 'MEGF6', 'PERM1', 'RNU1-7P', 'LINC01714', 'AL365255.1', 'AL139823.1', 'AL096855.2', 'LINC01786', 'TNFRSF14-AS1', 'LRRC47', 'PLEKHG5', 'MI